<a href="https://colab.research.google.com/github/girish445ai/Convolutional_Neural_Networks/blob/main/Convolutional_Neural_Networks_PART_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Part-B: Fine-tuning a pre-trained model

In [ ]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 181 kB 34.2 MB/s 
     |████████████████████████████████| 144 kB 36.8 MB/s 
     |████████████████████████████████| 63 kB 839 kB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=cf5bcd162714d2a2d46d2734bc8027eec2931281bde6b871ba6f0575588fc690
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
# Necessary packages.
import numpy as np
import pandas as pd
import os
import keras
import math
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import cv2
import wandb
from wandb.keras import WandbCallback

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet import ResNet50
from keras.applications.xception import Xception

In [ ]:
!curl -SL https://storage.googleapis.com/wandb_datasets/nature_12K.zip > nature_12K.zip
!unzip nature_12K.zip

train_datapath='inaturalist_12K/train/'
test_datapath='inaturalist_12K/val/'
categories=['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']

In [ ]:
import wandb
sweep_config = {
  'name': 'sweep1',  
  'method': 'grid',
  'metric': {
      'name': 'accuracy',
  },   
  'parameters': {
        
        
        'fc_size':{
            'values':[64]
        },
        'batch_norm':{
            'values':['yes']
        },
        'augmentation':{
            'values':['yes']   
        },
        'batch_size' :{
            'values':[32]
        },
        'droprate':{
            'values':[0.2]
        },
        'k_freeze_percent':{
            'values':[1]
        },
        'pre_train':{
            'values':['inceptionresnetv2','Xception','inceptionv3','resnet50']
        }

           
    }

}


In [ ]:
def PreTrain(config=sweep_config):

  #wandb init and log .

  wandb.init(config=config)
  config= wandb.init().config
  batch_norm=config['batch_norm']
  droprate=config['droprate']
  batch_size=config['batch_size']
  augmentation=config['augmentation']
  fc_size=config['fc_size']
  pre_train = config['pre_train']
  k_freeze_percent=config['k_freeze_percent']

  
  wandb.run.name = 'model_{}_fc_size_{}_dropout_{}_batchNorm_{}_dataAug_{}_k_freeze_percent_{}_batch_size_{}_'.format(pre_train, fc_size,droprate,batch_norm, augmentation,k_freeze_percent ,batch_size)
        

  if pre_train == 'inceptionv3':
    img_height = 299
    img_width = 299
    base_model = InceptionV3(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))
    # 'include_top = False ' leaves out the last layer as Image net has 1000 classes ,in our inaturalist dataset we have only 10 classes, we will add a dense layer of 10 classes .
    k = math.ceil(k_freeze_percent*311)  
    #There are 311 layers in inceptionv3 model . If 'k' is 70% then we want to Freeze the 70% of first layers in feature extraction ,use the weights of inceptionv3 model 
    # and train the rest of the 30% layers which are towards the end and update weights.  
                                                 

  if pre_train == 'inceptionresnetv2':
    img_height = 299
    img_width = 299
    base_model = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))
    k = math.ceil(k_freeze_percent*780)

  if pre_train == 'resnet50':
    img_height = 224
    img_width = 224
    base_model = ResNet50(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))
    k = math.ceil(k_freeze_percent*175)

  if pre_train == 'Xception':
    img_height = 299
    img_width = 299
    base_model = Xception(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))
    k = math.ceil(k_freeze_percent*132)  

  i = 0
  for layers in base_model.layers:
    layers.trainable = False
    i+=1
    if(i==k):
            break

  model = keras.Sequential([
      tf.keras.Input(shape=(img_height, img_width,3,)),
      base_model,
      Flatten(),
      Dense(fc_size,activation='relu'),
      
  ])
  if batch_norm == 'yes':
    model.add(BatchNormalization())
  model.add(Dropout(droprate))
  model.add(Dense(fc_size, activation='relu'))
  model.add(Dropout(droprate))
  model.add(Dense(10 ,activation='softmax'))
  

  #data generators for train(with and without validation) and validation
  augment= ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1.0 / 255,
    validation_split=0.1,
    dtype=tf.float32,
  )

  datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.1,
    dtype=tf.float32,
  )

  train_set = datagen.flow_from_directory(
    train_datapath,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='sparse',
    shuffle=True,
    subset='training',
    seed=123,
  )

  aug_set = augment.flow_from_directory(
      train_datapath,
      target_size=(img_height, img_width),
      batch_size=batch_size,
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      subset='training',
      seed=123,
  )

  val_set = datagen.flow_from_directory(
      train_datapath,
      target_size=(img_height, img_width),
      color_mode='rgb',
      class_mode='sparse',
      batch_size=batch_size,
      shuffle=True,
      subset='validation',
      seed=123,
  )

  model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=[tf.keras.losses.SparseCategoricalCrossentropy()],
    metrics=['accuracy'],
  )

  if augmentation =='no' :
    hist=model.fit(train_set,epochs=10,validation_data=val_set,callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 5), wandb.keras.WandbCallback()])
    
  
  else:
    hist=model.fit(aug_set,epochs=10,validation_data=val_set,callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 5), wandb.keras.WandbCallback()])

  val_acc=max(hist.history['val_accuracy'])
  params={'batchnorm':batch_norm,'augmentation':augmentation,'dropout':droprate,'pre_trained_model':pre_train,'val_acc':val_acc}
  wandb.log(params)

In [ ]:
sweep_id=wandb.sweep(sweep_config,project="Assignment2_partb", entity="girishrongali")
wandb.agent(sweep_id, PreTrain)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: slzk8zps
Sweep URL: https://wandb.ai/girishrongali/Assignment2_partb/sweeps/slzk8zps


wandb: Agent Starting Run: egp0yh44 with config:
wandb: 	augmentation: yes
wandb: 	batch_norm: yes
wandb: 	batch_size: 32
wandb: 	droprate: 0.4
wandb: 	fc_size: 128
wandb: 	k_freeze_percent: 0.8
wandb: 	pre_train: inceptionresnetv2
wandb: Currently logged in as: girishrongali (use `wandb login --relogin` to force relogin)


219070464/219055592 [==============================] - 2s 0us/step
Found 9000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/10
282/282 [==============================] - 435s 1s/step - loss: 1.3067 - accuracy: 0.5762 - val_loss: 1.2340 - val_accuracy: 0.7017 - _timestamp: 1648894901.0000 - _runtime: 456.0000
Epoch 2/10
282/282 [==============================] - 397s 1s/step - loss: 0.8552 - accuracy: 0.7434 - val_loss: 0.7139 - val_accuracy: 0.7738 - _timestamp: 1648895298.0000 - _runtime: 853.0000
Epoch 3/10
282/282 [==============================] - 393s 1s/step - loss: 0.7266 - accuracy: 0.7833 - val_loss: 0.6280 - val_accuracy: 0.8068 - _timestamp: 1648895691.0000 - _runtime: 1246.0000
Epoch 4/10
282/282 [==============================] - 386s 1s/step - loss: 0.6312 - accuracy: 0.8096 - val_loss: 0.6404 - val_accuracy: 0.8188 - _timestamp: 1648896080.0000 - _runtime: 1635.0000
Epoch 5/10
282/282 [=======

...

wandb: Agent Starting Run: 9pgmy4v6 with config:
wandb: 	augmentation: yes
wandb: 	batch_norm: yes
wandb: 	batch_size: 32
wandb: 	droprate: 0.4
wandb: 	fc_size: 128
wandb: 	k_freeze_percent: 0.8
wandb: 	pre_train: Xception


83697664/83683744 [==============================] - 1s 0us/step
Found 9000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/10
282/282 [==============================] - 397s 1s/step - loss: 1.2381 - accuracy: 0.6169 - val_loss: 1.2287 - val_accuracy: 0.6937 - _timestamp: 1648898994.0000 - _runtime: 407.0000
Epoch 2/10
282/282 [==============================] - 388s 1s/step - loss: 0.8386 - accuracy: 0.7494 - val_loss: 0.9433 - val_accuracy: 0.7397 - _timestamp: 1648899383.0000 - _runtime: 796.0000
Epoch 3/10
282/282 [==============================] - 388s 1s/step - loss: 0.7137 - accuracy: 0.7828 - val_loss: 0.7433 - val_accuracy: 0.7808 - _timestamp: 1648899769.0000 - _runtime: 1182.0000
Epoch 4/10
282/282 [==============================] - 387s 1s/step - loss: 0.6277 - accuracy: 0.8077 - val_loss: 0.8986 - val_accuracy: 0.7548 - _timestamp: 1648900159.0000 - _runtime: 1572.0000
Epoch 5/10
282/282 [=========

accuracy,▁▄▅▆▆▇▇███
dropout,▁
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▅▄▃▃▂▂▁▁▁
val_acc,▁
val_accuracy,▁▄▆▅▇▇▇█▇█
val_loss,█▄▂▄▁▁▂▁▃▂
accuracy,0.90022
augmentation,yes
batchnorm,yes
best_epoch,5


wandb: Agent Starting Run: rxfedq7z with config:
wandb: 	augmentation: yes
wandb: 	batch_norm: yes
wandb: 	batch_size: 32
wandb: 	droprate: 0.4
wandb: 	fc_size: 128
wandb: 	k_freeze_percent: 0.8
wandb: 	pre_train: inceptionv3


87924736/87910968 [==============================] - 1s 0us/step
Found 9000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/10
282/282 [==============================] - 359s 1s/step - loss: 1.2602 - accuracy: 0.6028 - val_loss: 0.8680 - val_accuracy: 0.7457 - _timestamp: 1648902972.0000 - _runtime: 372.0000
Epoch 2/10
282/282 [==============================] - 322s 1s/step - loss: 0.8893 - accuracy: 0.7282 - val_loss: 0.7785 - val_accuracy: 0.7698 - _timestamp: 1648903324.0000 - _runtime: 724.0000
Epoch 3/10
282/282 [==============================] - ETA: 0s - loss: 0.7592 - accuracy: 0.7691

## Evaluating the model on the TEST set 

In [ ]:
#this config gives best val accuracy
config_defaults={
   'fc_size':128,
   'batch_norm':'yes',
   'augmentation':'no',
    'droprate':0.4,
    'pre_train':'inceptionresnetv2',
    'k_freeze_percent':0.8,
    'batch_size':32
}

In [ ]:
def OptimalPreTrain(config=config_defaults):

  #wandb init and log commented to do trial run
  wandb.init()
  batch_norm=config['batch_norm']
  droprate=config['droprate']
  batch_size=config['batch_size']
  augmentation=config['augmentation']
  fc_size=config['fc_size']
  pre_train = config['pre_train']
  k_freeze_percent=config['k_freeze_percent']

  if pre_train == 'inceptionv3':
    img_height = 299
    img_width = 299
    base_model = InceptionV3(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))
    # 'include_top = False ' leaves out the last layer as Image net has 1000 classes ,in our inaturalist dataset we have only 10 classes, we will add a dense layer of 10 classes .
    k = math.ceil(k_freeze_percent*311)  
    #There are 311 layers in inceptionv3 model . If 'k' is 70% then we want to Freeze the 70% of first layers in feature extraction ,use the weights of inceptionv3 model 
    # and train the rest of the 30% layers which are towards the end and update weights.  
                                                 
  if pre_train == 'inceptionresnetv2':
    img_height = 299
    img_width = 299
    base_model = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))
    k = math.ceil(k_freeze_percent*780)

  if pre_train == 'resnet50':
    img_height = 224
    img_width = 224
    base_model = ResNet50(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))
    k = math.ceil(k_freeze_percent*175)

  if pre_train == 'Xception':
    img_height = 299
    img_width = 299
    base_model = Xception(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))
    k = math.ceil(k_freeze_percent*132)  

  i = 0
  for layers in base_model.layers:
    layers.trainable = False
    i+=1
    if(i==k):
            break

  model = keras.Sequential([
      tf.keras.Input(shape=(img_height, img_width,3,)),
      base_model,
      Flatten(),
      Dense(fc_size,activation='relu'),
      
  ])
  if batch_norm == 'yes':
    model.add(BatchNormalization())
  model.add(Dropout(droprate))
  model.add(Dense(fc_size, activation='relu'))
  model.add(Dropout(droprate))
  model.add(Dense(10 ,activation='softmax'))
  

  #data generators for train(with and without validation) and validation
  augment= ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1.0 / 255,
    validation_split=0.1,
    dtype=tf.float32,
  )

  datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.1,
    dtype=tf.float32,
  )

  train_set = datagen.flow_from_directory(
    train_datapath,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='sparse',
    shuffle=True,
    subset='training',
    seed=123,
  )

  aug_set = augment.flow_from_directory(
      train_datapath,
      target_size=(img_height, img_width),
      batch_size=batch_size,
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      subset='training',
      seed=123,
  )

  val_set = datagen.flow_from_directory(
      train_datapath,
      target_size=(img_height, img_width),
      color_mode='rgb',
      class_mode='sparse',
      batch_size=batch_size,
      shuffle=True,
      subset='validation',
      seed=123,
  )

  model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=[tf.keras.losses.SparseCategoricalCrossentropy()],
    metrics=['accuracy'],
  )

  if augmentation =='no' :
    hist=model.fit(train_set,epochs=10,validation_data=val_set,callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 5), wandb.keras.WandbCallback()])
    
  
  else:
    hist=model.fit(aug_set,epochs=10,validation_data=val_set,callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 5), wandb.keras.WandbCallback()])

  val_acc=max(hist.history['val_accuracy'])
  params={'batchnorm':batch_norm,'augmentation':augmentation,'dropout':droprate,'pre_trained_model':pre_train,'val_acc':val_acc}
  wandb.log(params)
  return model 

In [ ]:
model = OptimalPreTrain(config=config_defaults)

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Found 9000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/10
282/282 [==============================] - 328s 1s/step - loss: 1.1577 - accuracy: 0.6429 - val_loss: 3.7026 - val_accuracy: 0.5325 - _timestamp: 1648918415.0000 - _runtime: 344.0000
Epoch 2/10
282/282 [==============================] - 291s 1s/step - loss: 0.7046 - accuracy: 0.7862 - val_loss: 0.9555 - val_accuracy: 0.7377 - _timestamp: 1648918707.0000 - _runtime: 636.0000
Epoch 3/10
282/282 [==============================] - 288s 1s/step - loss: 0.5242 - accuracy: 0.8401 - val_loss: 1.6431 - val_accuracy: 0.7818 - _timestamp: 1648918998.0000 - _runtime: 927.0000
Epoch 4/10
282/282 [==============================] - 292s 1s/step - loss: 0.3872 - accuracy: 0.8833 - val_loss: 0.7329 - val_accuracy: 0.7718 - _timestamp: 1648919286.0000 - _runtime: 1215.0000
Epoch 5/10
282/282 [==============================] - 289s 1s/step - loss: 0.2869 - accuracy: 0

In [ ]:
# data generator for test set
test_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
  )

test_set = test_datagen.flow_from_directory(
      test_datapath,
      target_size=(299, 299),
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      seed=123,
  )

#evaluating model on test set
test_loss, test_acc = model.evaluate(test_set, verbose=2)
wandb.log({"test_loss": test_loss, "test_accuracy": test_acc})

Found 2000 images belonging to 10 classes.
63/63 - 50s - loss: 0.8487 - accuracy: 0.8115 - 50s/epoch - 787ms/step
